<a href="https://colab.research.google.com/github/ddecosmo-dev/thread-checker/blob/main/threadCheckerApplication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [49]:
#@title  fastener Classifier GUI
# ---------------------------------------------------------------------------------
# Section 1: Setup and Imports
# ---------------------------------------------------------------------------------
# Core libraries for machine learning and image processing
import tensorflow as tf
import numpy as np
from PIL import Image
import io
import json

# Libraries for building the interactive GUI in Colab
import ipywidgets as widgets
from ipywidgets import Layout
from google.colab import drive
from IPython.display import display

print("Libraries imported successfully!")

##constants
IMG_HEIGHT = 224
IMG_WIDTH = 224

# ---------------------------------------------------------------------------------
# Section 2: Connect to Google Drive
# ---------------------------------------------------------------------------------
# This will prompt you for authorization to access your Google Drive files.
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully!")
except Exception as e:
    print(f"Error mounting Google Drive: {e}")

#drive.mount("/content/drive", force_remount=True)

Libraries imported successfully!
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive mounted successfully!


In [50]:
#import data from training
# ---------------------------------------------------------------------------------
# Section 3: Configuration & Model Loading (from JSON)
# ---------------------------------------------------------------------------------
# --- IMPORTANT: UPDATE THESE TWO PATHS ---

# 1. Path to the JSON file containing your class structure map.
CLASS_MAP_PATH = '/content/drive/MyDrive/24-679 AI ML Projects/Project 0/class_map.json'

# 2. Path to your trained model file.
MODEL_PATH = '/content/drive/MyDrive/24-679 AI ML Projects/Project 0/thread_checker_model.keras'
# --- End of Configuration ---


In [51]:
# Load the class map from the JSON file
try:
    with open(CLASS_MAP_PATH, 'r') as f:
        class_map = json.load(f)
    # Create the flat CLASS_NAMES list for the model's output index.
    # It sorts the map by index (0, 1, 2...) and gets the corresponding key.
    CLASS_NAMES = [key for key, value in sorted(class_map.items(), key=lambda item: item[1])]
    print(f"Successfully loaded {len(CLASS_NAMES)} class names from JSON map.")
except FileNotFoundError:
    print(f"ERROR: Class map file not found at {CLASS_MAP_PATH}. Please create it first.")
    CLASS_NAMES = []
except Exception as e:
    print(f"An error occurred while loading the JSON map: {e}")
    CLASS_NAMES = []

Successfully loaded 3 class names from JSON map.


In [52]:
# Load the CNN model
model = None
if CLASS_NAMES: # Only proceed if the class map loaded successfully
    try:
        model = tf.keras.models.load_model(MODEL_PATH)
        print(f"✅ Model loaded successfully from {MODEL_PATH}")
    except Exception as e:
        print(f"🚨 ERROR: Could not load the model file at {MODEL_PATH}. {e}")

✅ Model loaded successfully from /content/drive/MyDrive/24-679 AI ML Projects/Project 0/thread_checker_model.keras
